In [ ]:
library(fs)
library(dplyr)
library(ggplot2)
library(stringr)
library(cowplot)
library(flowCore)
library(magrittr)

filter <- dplyr::filter

patient_df <- 
  read.csv(
    '~/Documents/Woodlist/Input_Flow_DFs/Jake_Masters_Cohort.csv')

patient_df <- 
  patient_df %>% 
  filter(Exclusion.Reason == 'Not Excluded')

patient_df <- patient_df[,!colnames(patient_df) == 'X']
    
patient_df %>% head()

In [ ]:
patient_df %>% count(CbioPortal.Cancer.Type)

In [ ]:
counts <- 
  patient_df %>% 
  count(CbioPortal.Cancer.Type) %>% 
  ungroup()


counts <-
  counts %>% 
  arrange(n) %>% 
  mutate(prop = n / sum(counts$n) * 100) %>%
  mutate(ypos = cumsum(prop)- 0.5*prop) %>% 
  mutate(n = factor(n, levels = sort(unique(n)))) 



colors <- 
  c("#c56179",
    "#5eb856",
    "#8662cb",
    "#b2b043",
    "#c95aaa",
    "#5d7b38",
    "#6d8cce",
    "#cd4d3b",
    "#50b69e",
    "#c18341")


p <- 
  counts %>% 
  ungroup() %>% 
  arrange(CbioPortal.Cancer.Type) %>% 
  ggplot(aes(x = '', y = prop, fill=CbioPortal.Cancer.Type)) + 
  geom_bar(stat = 'identity', width = 1)+ 
  geom_text(aes(y = ypos, label = n),
            color = "white", size = 5) +
  coord_polar('y', start = 0) +
  theme_void() + 
  theme(legend.title = element_blank()) 
  # scale_fill_manual(values = colors)


library(patchwork)


theme_border <- theme_light() + 
  theme(plot.background = element_rect(fill = 'white', colour = 'black', size = 1.75))

p <- p + 
  plot_annotation(caption = 'Breakdown of major cancer types represented by patients in the chosen dataset.', 
                 theme = theme_border)


ggsave('Piechart_by_disease_type.png', p)